In [ ]:
# !pip install --upgrade gensim
# !pip install "transformers==3.5.1"
# !pip install -U sentence-transformers
# !pip install kornia
# !pip install "torch==1.7.0"

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, hamming_loss, confusion_matrix
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data.sampler import SubsetRandomSampler
import transformers
from transformers import RobertaTokenizer, BertTokenizer, RobertaModel, BertModel, AdamW
from transformers import get_linear_schedule_with_warmup
import time
from kornia.losses import BinaryFocalLossWithLogits

import warnings
import traceback

In [ ]:
!nvidia-smi

Sat May 22 17:01:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 2080    Off  | 00000000:03:00.0 Off |                  N/A |
|  0%   42C    P0    42W / 260W |      3MiB /  7982MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 2080    Off  | 00000000:05:00.0 Off |                  N/A |
|  0%   

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda:0")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: GeForce RTX 2080


##Preparing Dataset

###Load dataset

In [ ]:
categories_number_words = {
        1: "Apoyo Pedagógico en asignaturas",
        3: "Apoyo pedagógico personal",
        4: "Tutoría entre pares",
        7: "Hacer a la familia partícipe del proceso",
        8: "Apoyo psicóloga(o)",
        9: "Apoyo fonoaudióloga(o)",
        10: "Apoyo Educador(a) Diferencial",
        11: "Apoyo Kinesióloga(o)",
        12: "Apoyo Médico General",
        13: "Apoyo Terapeuta Ocupacional",
        14: "Control Neurólogo",
        15: "Apoyo Interdisciplinario",
        16: "Adecuación curricular de acceso",
        17: "Adecuación curricular de objetivos"
    }
categories_words_number = {v: k for k, v in categories_number_words.items()}

diagnoses_codes = {
    "Trastorno específico del lenguaje": 0,
    "Trastorno por déficit atencional": 1,
    "Dificultad específica de aprendizaje": 2,
    "Discapacidad intelectual": 3,
    "Discapacidad visual": 4,
    "Trastorno del espectro autista": 5,
    "Discapacidad auditiva - Hipoacusia": 6,
    "Funcionamiento intelectual limítrofe": 7,
    "Síndrome de Down": 8,
    "Trastorno motor": 9,
    "Multidéficit": 10,
    "Retraso global del desarrollo": 11
}

diagnoses_keys = list(diagnoses_codes.keys())

def transform_diag_to_array(code):
    arr = np.zeros(len(diagnoses_keys), dtype=int)
    for (index, label) in enumerate(diagnoses_keys):
        if diagnoses_codes[label]==code:
            arr[index] = 1
    return arr

In [ ]:
train_dataset = pd.read_csv('/research/jamunoz/datasets/train_ds.csv', keep_default_na=False)
val_dataset = pd.read_csv('/research/jamunoz/datasets/val_ds.csv', keep_default_na=False)
test_dataset = pd.read_csv('/research/jamunoz/datasets/test_ds.csv', keep_default_na=False)
# train_dataset = pd.read_csv('gdrive/My Drive/magister/train_ds.csv', keep_default_na=False)
# val_dataset = pd.read_csv('gdrive/My Drive/magister/val_ds.csv', keep_default_na=False)
# test_dataset = pd.read_csv('gdrive/My Drive/magister/test_ds.csv', keep_default_na=False)


# Add OHE diagnosis
train_OHE_diags = []
for diag in train_dataset['Encoded Diagnosis']:
    train_OHE_diags.append(transform_diag_to_array(diag))
temp_train_diags_df = pd.DataFrame(train_OHE_diags, columns=diagnoses_keys)
train_dataset = pd.concat([train_dataset, temp_train_diags_df], axis=1)

val_OHE_diags = []
for diag in val_dataset['Encoded Diagnosis']:
    val_OHE_diags.append(transform_diag_to_array(diag))
temp_val_diags_df = pd.DataFrame(val_OHE_diags, columns=diagnoses_keys)
val_dataset = pd.concat([val_dataset, temp_val_diags_df], axis=1)

test_OHE_diags = []
for diag in test_dataset['Encoded Diagnosis']:
    test_OHE_diags.append(transform_diag_to_array(diag))
temp_test_diags_df = pd.DataFrame(test_OHE_diags, columns=diagnoses_keys)
test_dataset = pd.concat([test_dataset, temp_test_diags_df], axis=1)

In [ ]:
train_dataset.head()

In [ ]:
# y_keys = list(strat_present.keys())
Y_KEYS = list(categories_words_number.keys())

# df = pd.DataFrame(data=new_dataset_to_export)
# X = df
# Y = df[y_keys]
X_train = train_dataset.drop(Y_KEYS, axis=1)
Y_train = train_dataset[Y_KEYS]
X_val = val_dataset.drop(Y_KEYS, axis=1)
Y_val = val_dataset[Y_KEYS]
X_test = test_dataset.drop(Y_KEYS, axis=1)
Y_test = test_dataset[Y_KEYS]

strats_amounts = {
              'Adecuación curricular de acceso': 2264,
              'Hacer a la familia partícipe del proceso': 2048,
              'Apoyo Interdisciplinario': 1441, 
              'Apoyo Educador(a) Diferencial': 1311,
              'Apoyo pedagógico personal': 1240,
              'Apoyo fonoaudióloga(o)': 378,
              'Apoyo psicóloga(o)': 588,
              'Apoyo Terapeuta Ocupacional': 153,
              'Tutoría entre pares': 350,
              'Control Neurólogo': 63,
              'Apoyo Médico General': 64,
              'Apoyo Kinesióloga(o)': 32,
              'Adecuación curricular de objetivos': 281,
              'Apoyo Pedagógico en asignaturas': 1314
}
most_unbalanced_strategies = [strategy for strategy in Y_KEYS if (
    strats_amounts[strategy] < (len(X_train) + len(X_val) + len(X_test))*0.15 or strats_amounts[strategy] > (len(X_train) + len(X_val) + len(X_test))*0.85)]
less_unbalanced_strategies = [strategy for strategy in Y_KEYS if strategy not in most_unbalanced_strategies]
only_one_strat = [Y_KEYS[0]]

In [ ]:
Y_train.keys()

###Data explore

In [ ]:
print('Positive rates:')
for label in list(Y_train.keys()):
    print(label+': ', round(sum(Y_train[label])/len(Y_train[label]), 2))

Positive rates:
Apoyo Pedagógico en asignaturas:  0.43
Apoyo pedagógico personal:  0.41
Tutoría entre pares:  0.11
Hacer a la familia partícipe del proceso:  0.67
Apoyo psicóloga(o):  0.19
Apoyo fonoaudióloga(o):  0.12
Apoyo Educador(a) Diferencial:  0.43
Apoyo Kinesióloga(o):  0.01
Apoyo Médico General:  0.02
Apoyo Terapeuta Ocupacional:  0.05
Control Neurólogo:  0.02
Apoyo Interdisciplinario:  0.47
Adecuación curricular de acceso:  0.73
Adecuación curricular de objetivos:  0.09


###Dataset

In [ ]:
class AllJoinedObservationsDataset(Dataset):

    def __init__(self, data, tokenizer, chunk_len=200, overlap_len=50, approach="all"):
        self.tokenizer = tokenizer
        self.chunk_len = chunk_len
        self.overlap_len = overlap_len
        self.approach = approach
        self.data = data

    def __len__(self):
        return len(self.data)

    def long_terms_tokenizer(self, tokenized_data, labels):
        long_terms_token = []
        input_ids_list = []
        attention_mask_list = []
        token_type_ids_list = []
        labels_list = []

        previous_input_ids = tokenized_data["input_ids"].reshape(-1)
        previous_attention_mask = tokenized_data["attention_mask"].reshape(-1)
        previous_token_type_ids = tokenized_data["token_type_ids"].reshape(-1)
        remain = tokenized_data.get("overflowing_tokens").reshape(-1)
        labels = torch.tensor(labels, dtype=torch.int)

        input_ids_list.append(previous_input_ids)
        attention_mask_list.append(previous_attention_mask)
        token_type_ids_list.append(previous_token_type_ids)
        labels_list.append(labels)

        if remain.size() and self.approach != 'head':
            remain = torch.tensor(remain, dtype=torch.long)
            idxs = range(len(remain)+self.chunk_len)
            idxs = idxs[(self.chunk_len-self.overlap_len-2)
                         ::(self.chunk_len-self.overlap_len-2)]
            input_ids_first_overlap = previous_input_ids[-(
                self.overlap_len+1):-1]
            start_token = torch.tensor([101], dtype=torch.long)
            end_token = torch.tensor([102], dtype=torch.long)

            for i, idx in enumerate(idxs):
                if i == 0:
                    input_ids = torch.cat(
                        (input_ids_first_overlap, remain[:idx]))
                elif i == len(idxs):
                    input_ids = remain[idx:]
                elif previous_idx >= len(remain):
                    break
                else:
                    input_ids = remain[(previous_idx-self.overlap_len):idx]

                previous_idx = idx

                nb_token = len(input_ids)+2
                attention_mask = torch.ones(self.chunk_len, dtype=torch.long)
                attention_mask[nb_token:self.chunk_len] = 0
                token_type_ids = torch.zeros(self.chunk_len, dtype=torch.long)
                input_ids = torch.cat((start_token, input_ids, end_token))
                if self.chunk_len-nb_token > 0:
                    padding = torch.zeros(
                        self.chunk_len-nb_token, dtype=torch.long)
                    input_ids = torch.cat((input_ids, padding))

                input_ids_list.append(input_ids)
                attention_mask_list.append(attention_mask)
                token_type_ids_list.append(token_type_ids)
                labels_list.append(labels)
            if self.approach == "tail":
                input_ids_list = [input_ids_list[-1]]
                attention_mask_list = [attention_mask_list[-1]]
                token_type_ids_list = [token_type_ids_list[-1]]
                labels_list = [labels_list[-1]]

        return({
            'ids': input_ids_list,  # torch.tensor(ids, dtype=torch.long),
            # torch.tensor(mask, dtype=torch.long),
            'mask': attention_mask_list,
            # torch.tensor(token_type_ids, dtype=torch.long),
            'token_type_ids': token_type_ids_list,
            'labels': labels_list,
            'len': [torch.tensor(len(labels_list), dtype=torch.long)]
        })


    def __getitem__(self, idx):
      data_row = self.data.iloc[idx]
      labels =  data_row[Y_KEYS]

      data = self.tokenizer.encode_plus(
            data_row['All perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
      
      long_token = self.long_terms_tokenizer(data, labels)

      tensor_labels = torch.tensor(labels, dtype=torch.int)
      tensor_diags = torch.tensor(data_row[diagnoses_keys], dtype=torch.int)

      dict_to_return = dict(
          text_data = long_token,
          labels=tensor_labels,
          diagnostics=tensor_diags
      )

      return dict_to_return

In [ ]:
class SeparatedObservationsDataset(Dataset):

    def __init__(self, data, tokenizer, chunk_len=200, overlap_len=50, approach="all"):
        self.tokenizer = tokenizer
        self.chunk_len = chunk_len
        self.overlap_len = overlap_len
        self.approach = approach
        self.data = data

    def __len__(self):
        return len(self.data)

    def long_terms_tokenizer(self, tokenized_data, labels):
        long_terms_token = []
        input_ids_list = []
        attention_mask_list = []
        token_type_ids_list = []
        labels_list = []

        previous_input_ids = tokenized_data["input_ids"].reshape(-1)
        previous_attention_mask = tokenized_data["attention_mask"].reshape(-1)
        previous_token_type_ids = tokenized_data["token_type_ids"].reshape(-1)
        remain = tokenized_data.get("overflowing_tokens").reshape(-1)
        labels = torch.tensor(labels, dtype=torch.int)

        input_ids_list.append(previous_input_ids)
        attention_mask_list.append(previous_attention_mask)
        token_type_ids_list.append(previous_token_type_ids)
        labels_list.append(labels)

        if remain.size() and self.approach != 'head':
            remain = torch.tensor(remain, dtype=torch.long)
            idxs = range(len(remain)+self.chunk_len)
            idxs = idxs[(self.chunk_len-self.overlap_len-2)
                         ::(self.chunk_len-self.overlap_len-2)]
            input_ids_first_overlap = previous_input_ids[-(
                self.overlap_len+1):-1]
            start_token = torch.tensor([101], dtype=torch.long)
            end_token = torch.tensor([102], dtype=torch.long)

            for i, idx in enumerate(idxs):
                if i == 0:
                    input_ids = torch.cat(
                        (input_ids_first_overlap, remain[:idx]))
                elif i == len(idxs):
                    input_ids = remain[idx:]
                elif previous_idx >= len(remain):
                    break
                else:
                    input_ids = remain[(previous_idx-self.overlap_len):idx]

                previous_idx = idx

                nb_token = len(input_ids)+2
                attention_mask = torch.ones(self.chunk_len, dtype=torch.long)
                attention_mask[nb_token:self.chunk_len] = 0
                token_type_ids = torch.zeros(self.chunk_len, dtype=torch.long)
                input_ids = torch.cat((start_token, input_ids, end_token))
                if self.chunk_len-nb_token > 0:
                    padding = torch.zeros(
                        self.chunk_len-nb_token, dtype=torch.long)
                    input_ids = torch.cat((input_ids, padding))

                input_ids_list.append(input_ids)
                attention_mask_list.append(attention_mask)
                token_type_ids_list.append(token_type_ids)
                labels_list.append(labels)
            if self.approach == "tail":
                input_ids_list = [input_ids_list[-1]]
                attention_mask_list = [attention_mask_list[-1]]
                token_type_ids_list = [token_type_ids_list[-1]]
                labels_list = [labels_list[-1]]

        return({
            'ids': input_ids_list,  # torch.tensor(ids, dtype=torch.long),
            # torch.tensor(mask, dtype=torch.long),
            'mask': attention_mask_list,
            # torch.tensor(token_type_ids, dtype=torch.long),
            'token_type_ids': token_type_ids_list,
            'labels': labels_list,
            'len': [torch.tensor(len(labels_list), dtype=torch.long)]
        })


    def __getitem__(self, idx):
      data_row = self.data.iloc[idx]
      labels =  data_row[Y_KEYS]

      set_data = self.tokenizer.encode_plus(
            data_row['Special Education Teacher Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
    
      st_data = self.tokenizer.encode_plus(
            data_row['Speech Therapist Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
        
      p_data = self.tokenizer.encode_plus(
            data_row['Psychologist Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
        
      m_data = self.tokenizer.encode_plus(
            data_row['Medical Perceptions'],
            max_length=self.chunk_len,
            pad_to_max_length=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_overflowing_tokens=True,
            return_tensors='pt')
      
      set_long_token = self.long_terms_tokenizer(set_data, labels)      
      st_long_token = self.long_terms_tokenizer(st_data, labels)      
      p_long_token = self.long_terms_tokenizer(p_data, labels)      
      m_long_token = self.long_terms_tokenizer(m_data, labels)

      tensor_labels = torch.tensor(labels, dtype=torch.int)
      tensor_diags = torch.tensor(data_row[diagnoses_keys], dtype=torch.int)

      dict_to_return = dict(
          text_data_set = set_long_token,
          text_data_st = st_long_token,
          text_data_p = p_long_token,
          text_data_m = m_long_token,
          labels=tensor_labels,
          diagnostics=tensor_diags
      )

      return dict_to_return

###Dataloaders

In [ ]:
TRAIN_BATCH_SIZE=1
CHUNK_LEN=150
OVERLAP_LEN=30

print('Loading BERT tokenizer...')
bert_tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", do_lower_case=True)

def my_collate1(batches):
  modified_batches = []
  for batch in batches:
    batch_dict = {}
    for key, value in batch.items():
      if key == "text_data":
        new_dict = {}
        for sub_key, sub_value in batch[key].items():
          new_dict[sub_key] = torch.stack(sub_value)
        batch_dict[key] = new_dict
      else:
        batch_dict[key] = value
    modified_batches.append(batch_dict)
  return modified_batches

transformed_train_dataset=AllJoinedObservationsDataset(
    train_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

transformed_val_dataset=AllJoinedObservationsDataset(
    val_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

transformed_test_dataset=AllJoinedObservationsDataset(
    test_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

train_data_loader=DataLoader(
    transformed_train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

val_data_loader=DataLoader(
    transformed_val_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

test_data_loader=DataLoader(
    transformed_test_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate1)

############ separated observations ############

def my_collate2(batches):
  modified_batches = []
  for batch in batches:
    batch_dict = {}
    for key, value in batch.items():
      if key == "text_data_set" or key == "text_data_st" or key == "text_data_p" or key == "text_data_m":
        new_dict = {}
        for sub_key, sub_value in batch[key].items():
          new_dict[sub_key] = torch.stack(sub_value)
        batch_dict[key] = new_dict
      else:
        batch_dict[key] = value
    modified_batches.append(batch_dict)
  return modified_batches

divided_transformed_train_dataset=SeparatedObservationsDataset(
    train_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

divided_transformed_val_dataset=SeparatedObservationsDataset(
    val_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

divided_transformed_test_dataset=SeparatedObservationsDataset(
    test_dataset,
    tokenizer=bert_tokenizer,
    chunk_len=CHUNK_LEN,
    overlap_len=OVERLAP_LEN)

divided_train_data_loader=DataLoader(
    divided_transformed_train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate2)

divided_val_data_loader=DataLoader(
    divided_transformed_val_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate2)

divided_test_data_loader=DataLoader(
    divided_transformed_test_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    collate_fn=my_collate2)

Loading BERT tokenizer...


###Fine-tunning BERT

In [ ]:
class Bert_FT_Model(nn.Module):
    """ A Model for bert fine tuning """

    def __init__(self, n_classes):
        super(Bert_FT_Model, self).__init__()
        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, n_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids):
        """ Define how to perfom each call
        Parameters
        __________
        ids: array
            -
        mask: array
            - 
        token_type_ids: array
            -
        Returns
        _______
            - 
        """
        _, output = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)
        return output
    
    
class Single_Bert_FT_Model(nn.Module):
    """ A Model for bert fine tuning """

    def __init__(self):
        super(Single_Bert_FT_Model, self).__init__()
        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, 1)
        # self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids, is_eval=False):
        """ Define how to perfom each call
        Parameters
        __________
        ids: array
            -
        mask: array
            - 
        token_type_ids: array
            -
        Returns
        _______
            - 
        """
        _, output = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        if is_eval:
          print('representacion: ', output)
        # output = self.relu(output)
        # output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)
        return output

In [ ]:
class Bert_Hierarchical_Model(nn.Module):

    def __init__(self, n_classes, pooling_method="mean"):
        super(Bert_Hierarchical_Model, self).__init__()

        self.pooling_method = pooling_method

        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, n_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids, lengt):

        _, pooled_out = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)

        chunks_emb = pooled_out.split_with_sizes(lengt)

        if self.pooling_method == "mean":
            emb_pool = torch.stack([torch.mean(x, 0) for x in chunks_emb])
        elif self.pooling_method == "max":
            emb_pool = torch.stack([torch.max(x, 0)[0] for x in chunks_emb])

        output = self.relu(emb_pool)
        output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)

        return output
    

class Single_Bert_Hierarchical_Model(nn.Module):

    def __init__(self, pooling_method="mean"):
        super(Single_Bert_Hierarchical_Model, self).__init__()

        self.pooling_method = pooling_method

        self.bert_path = "dccuchile/bert-base-spanish-wwm-cased"
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.out = nn.Linear(768, 1)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.3)

    def forward(self, ids, mask, token_type_ids, lengt):

        _, pooled_out = self.bert(
            ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)

        chunks_emb = pooled_out.split_with_sizes(lengt)

        if self.pooling_method == "mean":
            output = torch.stack([torch.mean(x, 0) for x in chunks_emb])
        elif self.pooling_method == "max":
            output = torch.stack([torch.max(x, 0)[0] for x in chunks_emb])

#         output = self.relu(output)
#         output = self.dropout(output)
        output = self.out(output)
        # output = torch.sigmoid(output)

        return output

In [ ]:
import statistics

def get_results(targets, outputs):
  TN = 0
  TP = 0
  FP = 0
  FN = 0
  for (i, output) in enumerate(outputs):
    if output==0:
      if targets[i]==0:
        TN += 1
      else:
        FN += 1
    else:
      if targets[i]==1:
        TP += 1
      else:
        FP += 1
  return TP, TN, FP, FN

def findMinDiff(arr):
    n = len(arr)
    arr = sorted(arr)
    diff = 0.5
    for i in range(n-1):
        if arr[i+1] - arr[i] > 0 and arr[i+1] - arr[i] < diff:
            diff = arr[i+1] - arr[i]
    return diff

def get_thresholds(targets, outputs):
  best_thresholds = []
  for i in range(len(outputs[0])):
    real_preds = outputs[:, i]
    trues = targets[:, i]
    max_g = 0
#     max_f1 = 0
    delta_threshold = 0.0001 # findMinDiff(real_preds)*0.9
    positive_ratio = sum(trues)/len(trues)
#     print('pr: ', positive_ratio)
    if positive_ratio > 0.6:
      local_best = 0
      curr_threshold = min(real_preds)
#       print(curr_threshold)
      while curr_threshold < 1:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
#         f1 = f1_score(trues, preds)
#         print(f1, max_f1, curr_threshold, local_best, tp, tn, fp, fn)
        if tp < tn:
          break
        if g_mean > max_g:
#         if f1 > max_f1:
          max_g = g_mean
#           max_f1 = f1
          local_best = curr_threshold
        curr_threshold += delta_threshold
      best_thresholds.append(local_best)
    elif positive_ratio < 0.4:
      local_best = 1
      curr_threshold = max(real_preds)
      while curr_threshold > 0:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
#         f1 = f1_score(trues, preds)
        if tn < tp:
          break
        if g_mean > max_g:
#         if f1 > max_f1:
          max_g = g_mean
#           max_f1 = f1
          local_best = curr_threshold
        curr_threshold -= delta_threshold
      best_thresholds.append(local_best)
    else:
      local_best = 0.5
      best_thresholds.append(local_best)
  return best_thresholds

def get_individual_threshold(target, output, mode='g_mean'):
    real_preds = output
    trues = target
    max_g = 0
    max_f1 = 0
    delta_threshold = 0.0001 # findMinDiff(real_preds)*0.9
    positive_ratio = sum(trues)/len(trues)
#     print('pr: ', positive_ratio)
    if positive_ratio > 0.5:
      local_best = 0
      curr_threshold = min(real_preds)
#       print(curr_threshold)
      while curr_threshold < 1:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
        f1 = f1_score(trues, preds)
#         print(f1, max_f1, curr_threshold, local_best, tp, tn, fp, fn)
        if tp < tn:
          break
        if mode == 'g_mean':
            if g_mean > max_g:
                max_g = g_mean
                local_best = curr_threshold
        elif mode == 'f1':
            if f1 > max_f1:
                max_f1 = f1
                local_best = curr_threshold
        curr_threshold += delta_threshold
      return local_best
    else:
      local_best = 1
      curr_threshold = max(real_preds)
      while curr_threshold > 0:
        preds = [1 if pred > curr_threshold else 0 for pred in real_preds]
        tp, tn, fp, fn = get_results(trues, preds)
        recall = tp/(tp+fn)
        specificity = tn/(tn+fp)
        g_mean = np.sqrt(recall*specificity)
        f1 = f1_score(trues, preds)
        if tn < tp:
          break
        if mode == 'g_mean':
            if g_mean > max_g:
                max_g = g_mean
                local_best = curr_threshold
        elif mode == 'f1':
            if f1 > max_f1:
                max_f1 = f1
                local_best = curr_threshold
        curr_threshold -= delta_threshold
      return local_best
    # else:
    #   local_best = 0.5
    #   return local_best
  

In [ ]:
def loss_fun(outputs, targets):
    loss = nn.BCEWithLogitsLoss()
    # loss = BinaryFocalLossWithLogits(alpha=0.25, reduction='mean')
    try:
      return loss(outputs, targets)
    except Exception:
      print(outputs, targets)
      traceback.print_exc()
    # return nn.BCEWithLogitsLoss()(outputs, targets)

def individual_evaluation_1(target, predicted):
  individual = {}
  for i in range(len(target[0])):
    temp_t = target[:, i]
    temp_p = predicted[:, i]
    diction = dict(
        accuracy=accuracy_score(temp_t, temp_p),
        f1=f1_score(temp_t, temp_p)
    )
    individual[str(i)] = diction
  return individual

def evaluate(target, predicted):
    thresholds = get_thresholds(target, predicted)
    print('thresholds: ', thresholds)
    true_predicted = np.array([[1 if val > thresholds[i] else 0 for (i, val) in enumerate(pred)] for pred in predicted])
    accuracy = accuracy_score(target, true_predicted)
    macro_f1 = f1_score(target, true_predicted, average='macro')
    micro_f1 = f1_score(target, true_predicted, average='micro')
    weighted_f1 = f1_score(target, true_predicted, average='weighted')
    hl = hamming_loss(target, true_predicted)
    js = jaccard_score(target, true_predicted)
    macro_js = jaccard_score(target, true_predicted, average="macro")
    micro_js = jaccard_score(target, true_predicted, average="micro")
    individual = individual_evaluation_1(target, true_predicted)
    return {
        "accuracy": accuracy,
        "jaccard_score_average": js,
        "jaccard_score_macro": macro_js,
        "jaccard_score_micro": micro_js,
        "macro-f1": macro_f1,
        "micro-f1": micro_f1,
        "Hamming Loss": hl,
        "Individual": individual
    }

def individual_evaluation(target, predicted):
    threshold_g = get_individual_threshold(target, predicted)
    print('threshold (g-mean): ',threshold_g)
    true_predicted_g = np.array([1 if val > threshold_g else 0 for val in predicted])
    accuracy_g = accuracy_score(target, true_predicted_g)
    f1_g = f1_score(target, true_predicted_g)
    tp, tn, fp, fn = get_results(target, true_predicted_g)
    recall_g = tp/(tp+fn)
    specificity_g = tn/(tn+fp)
    g_mean_g = np.sqrt(recall_g*specificity_g)
    pr = sum(target)/len(target)
    
    threshold_f = get_individual_threshold(target, predicted, mode='f1')
    print('threshold (f1): ',threshold_f)
    true_predicted_f = np.array([1 if val > threshold_f else 0 for val in predicted])
    accuracy_f = accuracy_score(target, true_predicted_f)
    f1_f = f1_score(target, true_predicted_f)
    tp, tn, fp, fn = get_results(target, true_predicted_f)
    recall_f = tp/(tp+fn)
    specificity_f = tn/(tn+fp)
    g_mean_f = np.sqrt(recall_f*specificity_f)

    default_true_predicted = np.array([1 if val > 0.5 else 0 for val in predicted])
    default_accuracy = accuracy_score(target, default_true_predicted)
    default_f1 = f1_score(target, default_true_predicted)
    tp, tn, fp, fn = get_results(target, default_true_predicted)
    default_recall = tp/(tp+fn)
    default_specificity = tn/(tn+fp)
    default_g_mean = np.sqrt(default_recall*default_specificity)
    return {
        "Positive Rate": pr,
        "max g_mean metrics": {
            "threshold": threshold_g,
            "accuracy": accuracy_g,
            "f1": f1_g,
            "recall": recall_g,
            "specificity": specificity_g,
            "g_mean": g_mean_g},
        "max f1 metrics": {
            "threshold": threshold_f,
            "accuracy": accuracy_f,
            "f1": f1_f,
            "recall": recall_f,
            "specificity": specificity_f,
            "g_mean": g_mean_f},
        "default_accuracy": default_accuracy,
        "default_f1": default_f1,
        "default_recall": default_recall,
        "default_specificity": default_specificity,
        "default_g_mean": default_g_mean
    }

In [ ]:
def train_loop_fun1(data_loader, model, optimizer, device, grad_accs, scheduler=None):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["text_data"]["labels"] for data in batch]
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        loss = loss_fun(outputs, targets)
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses


def individual_train_loop_fun1(data_loader, model, optimizer, device, grad_accs, scheduler=None, label_index=0):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["text_data"]["labels"][:, label_index] for data in batch]
        targets = []
        for label_set in labels:
          miniset = []
          for label in label_set:
            miniset.append(torch.tensor([label]))
          targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
        # outputs = torch.cat(torch.unbind(outputs))
        loss = loss_fun(outputs, targets)
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses

In [ ]:
def eval_loop_fun1(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["text_data"]["labels"] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

def individual_eval_loop_fun1(data_loader, model, device, label_index=0):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["text_data"]["labels"][:, label_index] for data in batch]
        targets = []
        for label_set in labels:
          miniset = []
          for label in label_set:
            miniset.append(torch.tensor([label]))
          targets.append(torch.stack(miniset))

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids)
            # outputs = torch.cat(torch.unbind(outputs))
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    # print(fin_outputs)
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

In [ ]:
def train_loop_fun2(data_loader, model, optimizer, device, grad_accs, scheduler=None):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["labels"] for data in batch]
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.stack(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
        loss = loss_fun(outputs, targets)
        # print('calcula la loss', loss)
        # Gradient accumulation loss
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses

def individual_train_loop_fun2(data_loader, model, optimizer, device, grad_accs, scheduler=None, label_index=0):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor([labels[0]])]
            targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
        # outputs = torch.cat(torch.unbind(outputs))
        loss = loss_fun(outputs, targets)
        (loss / grad_accs).backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} seconds ___")
            t0 = time.time()
    return losses

In [ ]:
def eval_loop_fun2(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        #         model.half()
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        targets = [data["labels"] for data in batch]
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.stack(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

def individual_eval_loop_fun2(data_loader, model, device, label_index=0):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor([labels[0]])]
            targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        with torch.no_grad():
            outputs = model(ids=ids, mask=mask, token_type_ids=token_type_ids, lengt=lengt)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        outputs = torch.sigmoid(outputs)
        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

In [ ]:
class Individual_RoBERT_Model(nn.Module):
    """ Make an LSTM model over a fine tuned bert model.
    Parameters
    __________
    bertFineTuned: BertModel
        A bert fine tuned instance
    n_classes: int
        Number of classes
    """

    def __init__(self, bertFineTuned, n_diags, n_layers=1):
        super(Individual_RoBERT_Model, self).__init__()
        self.bertFineTuned = bertFineTuned
        self.lstm = nn.LSTM(768, 100, num_layers=n_layers, bidirectional=True)
        self.out = nn.Linear(100 + n_diags, 1)

    def forward(self, ids, mask, token_type_ids, lengt, diagnostics):
        """ Define how to performed each call
        Parameters
        __________
        ids: array
            -
        mask: array
            - 
        token_type_ids: array
            -
        lengt: int
            -
        Returns:
        _______
        -
        """
        _, pooled_out = self.bertFineTuned(
            ids, attention_mask=mask, token_type_ids=token_type_ids)
        chunks_emb = pooled_out.split_with_sizes(lengt)

        seq_lengths = torch.LongTensor([x for x in map(len, chunks_emb)])

        batch_emb_pad = nn.utils.rnn.pad_sequence(
            chunks_emb, padding_value=-91, batch_first=True)
        batch_emb = batch_emb_pad.transpose(0, 1)  # (B,L,D) -> (L,B,D)
        lstm_input = nn.utils.rnn.pack_padded_sequence(
            batch_emb, seq_lengths.cpu().numpy(), batch_first=False, enforce_sorted=False)

        packed_output, (h_t, h_c) = self.lstm(lstm_input, )  # (h_t, h_c))
#         output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, padding_value=-91)

        h_t = h_t.view(-1, 100)
        output = torch.cat((h_t[0], diagnostics), dim=0)
        output = self.out(output)
        # output = torch.sigmoid(output)

        return output

In [ ]:
def individual_rnn_train_loop_fun1(data_loader, model, optimizer, device, grad_accs, scheduler=None, label_index=0):
    model.train()
    t0 = time.time()
    losses = []
    optimizer.zero_grad()
    for batch_idx, batch in enumerate(data_loader):
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor(labels[0])]
            targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]
        diagnostics = [data["diagnostics"] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]
        diagnostics = torch.cat(diagnostics)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        diagnostics = diagnostics.to(device, dtype=torch.long)
        outputs = model(ids=ids, mask=mask,
                        token_type_ids=token_type_ids, lengt=lengt, diagnostics=diagnostics)
        loss = loss_fun(outputs, targets)
        (loss / grad_accs).backward()
        # loss.backward()
        model.float()
        if (batch_idx + 1) % grad_accs == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
        losses.append(loss.item())
        if batch_idx % 250 == 0:
            print(
                f"___ batch index = {batch_idx} / {len(data_loader)} ({100*batch_idx / len(data_loader):.2f}%), loss = {np.mean(losses[-10:]):.4f}, time = {time.time()-t0:.2f} secondes ___")
            t0 = time.time()
    return losses


def individual_rnn_eval_loop_fun1(data_loader, model, device, label_index=0):
    model.eval()
    fin_targets = []
    fin_outputs = []
    losses = []
    for batch_idx, batch in enumerate(data_loader):
        
        ids = [data["text_data"]["ids"] for data in batch]
        mask = [data["text_data"]["mask"] for data in batch]
        token_type_ids = [data["text_data"]["token_type_ids"] for data in batch]
        labels = [data["labels"][label_index] for data in batch]
        targets = []
        if len(labels) > 1:
            for label_set in labels:
              miniset = []
              for label in label_set:
                miniset.append(torch.tensor([label]))
              targets.append(torch.stack(miniset))
        else:
            miniset = [torch.tensor(labels[0])]
            targets.append(torch.stack(miniset))
        lengt = [data["text_data"]['len'] for data in batch]
        diagnostics = [data["diagnostics"] for data in batch]

        ids = torch.cat(ids)
        mask = torch.cat(mask)
        token_type_ids = torch.cat(token_type_ids)
        targets = torch.cat(targets)
        lengt = torch.cat(lengt)
        lengt = [x.item() for x in lengt]
        diagnostics = torch.cat(diagnostics)

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        diagnostics = diagnostics.to(device, dtype=torch.long)
        with torch.no_grad():
            outputs = model(ids=ids, mask=mask,
                            token_type_ids=token_type_ids, lengt=lengt, diagnostics=diagnostics)
            loss = loss_fun(outputs, targets)
            losses.append(loss.item())

        fin_targets.append(targets.cpu().detach().numpy())
        fin_outputs.append(outputs.cpu().detach().numpy())
    return np.concatenate(fin_outputs), np.concatenate(fin_targets), losses

In [ ]:
# # label
# multi_labs = [0]
# learning_rates_coeffs = {
#     0: 5,
#     1: 10,
#     2: 5,
#     3: 1,
#     4: 5,
#     5: 1,
#     6: 5,
#     7: 1,
#     8: 1,
#     9: 1,
#     10: 1,
#     11: 10,
#     12: 1,
#     13: 5
# }

G_A =[16]
N_L = [1] # , 3, 5]  # , 7]
lrs = [0.1, 1]
EPOCH = 8

multi_labs = [7, 8, 9, 10, 11, 12, 13]

for lab in multi_labs:
    label = lab
    print('#################################################### Para label: ', Y_KEYS[label], " ("+str(label)+") ##############")
    best_model = None
    best_f1 = 0
    best_threshold = 0
    

    model=torch.load("/research/jamunoz/models/fine_tuned_bert/model_"+str(label)+".pt")
    bert_model = list(model.children())[0]
#     bert_path = "dccuchile/bert-base-spanish-wwm-cased"
#     bert_model = transformers.BertModel.from_pretrained(bert_path)

    # freeze fine tuner bert
    for param in bert_model.parameters():
        param.requires_grad = False

    for l_r in lrs:
      lr=l_r*1e-6 # learning_rates_coeffs[lab]*1e-6#1e-3
      num_training_steps=int(len(transformed_train_dataset) / TRAIN_BATCH_SIZE * EPOCH)
      print("--------------------------------------lr: "+str(lr)+"-----------------------------")
      for n_lay in N_L:
        print("+++++++++++++++++++++++++++++++++++++++++++++++++++ n layers: "+str(n_lay)+" ++++++++++++++++++++++++++++++++++++++++++++++++")
        for grad_acc in G_A:
          print("/////////////////////////////////////////// GA: "+str(grad_acc)+" ///////////////////////////////////////////////////////")

          # model=torch.load("gdrive/My Drive/magister/models/fine_tuned_bert/model_"+str(label)+".pt", map_location=device)
          
              
          model_rnn=Individual_RoBERT_Model(bertFineTuned=bert_model, n_diags=len(diagnoses_keys), n_layers=n_lay).to(device)
          optimizer=AdamW(model_rnn.parameters(), lr=lr)
          scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                  num_warmup_steps=0,
                                                  num_training_steps=num_training_steps)

          val_losses=[]
          batches_losses=[]
          val_acc=[]
          for epoch in range(EPOCH):
              t0 = time.time()    
              print(f"\n=============== EPOCH {epoch+1} / {EPOCH} ===============\n")
              batches_losses_tmp=individual_rnn_train_loop_fun1(train_data_loader, model_rnn, optimizer, device, grad_acc, scheduler=scheduler, label_index=label)
              epoch_loss=np.mean(batches_losses_tmp)
              print(f"\n*** avg_loss : {epoch_loss:.2f}, time : ~{(time.time()-t0)//60} min ({time.time()-t0:.2f} sec) ***\n")
              t1=time.time()
              output, target, val_losses_tmp=individual_rnn_eval_loop_fun1(val_data_loader, model_rnn, device)
              print(f"==> evaluation : avg_loss = {np.mean(val_losses_tmp):.2f}, time : {time.time()-t1:.2f} sec\n")
              tmp_evaluate=individual_evaluation(target, output)
              print(f"=====>\t{tmp_evaluate}")
              if tmp_evaluate['max f1 metrics']['f1'] > best_f1:
                    del best_model
                    torch.cuda.empty_cache()
                    best_model = model_rnn
                    best_f1 = tmp_evaluate['max f1 metrics']['f1']
                    best_threshold = tmp_evaluate['max f1 metrics']['threshold']
              val_acc.append(tmp_evaluate['default_accuracy'])
              val_losses.append(val_losses_tmp)
              batches_losses.append(batches_losses_tmp)
              
          del model_rnn, optimizer, scheduler
          torch.cuda.empty_cache()
          print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    torch.save(best_model, f"/research/jamunoz/models/robert_lin/model_"+str(lab)+"_tr_"+str(best_threshold)+".pt")
    del model, bert_model, best_model
    torch.cuda.empty_cache()
    print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx END Para label: ", Y_KEYS[label], " ("+str(label)+") xxxxxxxxxxxxxxxxxxxxxxxxx")
    
#     print("\t§§ the RNN model has been saved §§")
#     torch.save(model_rnn, f"/research/jamunoz/models/robert/model_rnn_epoch{epoch+1}.pt")

#################################################### Para label:  Apoyo Kinesióloga(o)  (7) ##############


--------------------------------------lr: 1e-07-----------------------------
+++++++++++++++++++++++++++++++++++++++++++++++++++ n layers: 1 ++++++++++++++++++++++++++++++++++++++++++++++++
/////////////////////////////////////////// GA: 16 ///////////////////////////////////////////////////////


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



=============== EPOCH 1 / 8 ===============

___ batch index = 0 / 1836 (0.00%), loss = 0.7341, time = 0.04 secondes ___


/home/jamunoz/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2016: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-11-5fb9fe6293fb>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  remain = torch.tensor(remain, dtype=torch.long)
<ipython-input-23-5db6513a9937>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires

...

___ batch index = 1500 / 1836 (81.70%), loss = 0.5065, time = 11.92 secondes ___


___ batch index = 1750 / 1836 (95.32%), loss = 0.4644, time = 13.04 secondes ___



*** avg_loss : 0.55, time : ~1.0 min (91.23 sec) ***



==> evaluation : avg_loss = 0.70, time : 29.29 sec

threshold (g-mean):  1
threshold (f1):  1
=====>	{'Positive Rate': 0.4332784184514003, 'max g_mean metrics': {'threshold': 1, 'accuracy': 0.5667215815485996, 'f1': 0.0, 'recall': 0.0, 'specificity': 1.0, 'g_mean': 0.0}, 'max f1 metrics': {'threshold': 1, 'accuracy': 0.5667215815485996, 'f1': 0.0, 'recall': 0.0, 'specificity': 1.0, 'g_mean': 0.0}, 'default_accuracy': 0.5667215815485996, 'default_f1': 0.0, 'default_recall': 0.0, 'default_specificity': 1.0, 'default_g_mean': 0.0}
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx END Para label:  Adecuación curricular de objetivos  (13) xxxxxxxxxxxxxxxxxxxxxxxxx


In [ ]:
# label
# G_8, L_1, lr_1, f1_0.44
# G_8, L_1, lr_5, f1_0.04
# G_8, L_3, lr_1, f1_0.07
# G_8, L_3, lr_5, f1_0.07

EPOCH = 6

G_A =[16]
N_L = [1] # , 3, 5]  # , 7]
lrs = [0.1, 1]
GRADIENT_ACCUMULATIONS = 16
N_LAYERS = 3


multi_labs = [7, 8, 9, 10, 11, 12, 13]
learning_rates_coeffs = {
    0: 5,
    1: 10,
    2: 5,
    3: 1,
    4: 5,
    5: 1,
    6: 5,
    7: 1,
    8: 1,
    9: 1,
    10: 1,
    11: 10,
    12: 1,
    13: 5
}

for lab in multi_labs:
    label = lab
    print('#################################################### Para label: ', Y_KEYS[label], " ("+str(label)+") ##############")
    best_model = None
    best_f1 = 0
    best_threshold = 0
    
    model=torch.load("/research/jamunoz/models/average_fine_tuned_bert/model_"+str(label)+".pt")
    bert_model = list(model.children())[0]

    # freeze fine tuner bert
    for param in bert_model.parameters():
        param.requires_grad = False
        
    for l_r in lrs:
      lr=l_r*1e-6 # learning_rates_coeffs[lab]*1e-6#1e-3
      num_training_steps=int(len(transformed_train_dataset) / TRAIN_BATCH_SIZE * EPOCH)
      print("--------------------------------------lr: "+str(lr)+"-----------------------------")
      for n_lay in N_L:
        print("+++++++++++++++++++++++++++++++++++++++++++++++++++ n layers: "+str(n_lay)+" ++++++++++++++++++++++++++++++++++++++++++++++++")
        for grad_acc in G_A:
          print("/////////////////////////////////////////// GA: "+str(grad_acc)+" ///////////////////////////////////////////////////////")

          # model=torch.load("gdrive/My Drive/magister/models/fine_tuned_bert/model_"+str(label)+".pt", map_location=device)
              
          model_rnn=Individual_RoBERT_Model(bertFineTuned=bert_model, n_diags=len(diagnoses_keys), n_layers=n_lay).to(device)
          optimizer=AdamW(model_rnn.parameters(), lr=lr)
          scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                  num_warmup_steps=0,
                                                  num_training_steps=num_training_steps)

          val_losses=[]
          batches_losses=[]
          val_acc=[]
          for epoch in range(EPOCH):
              t0 = time.time()    
              print(f"\n=============== EPOCH {epoch+1} / {EPOCH} ===============\n")
              batches_losses_tmp=individual_rnn_train_loop_fun1(train_data_loader, model_rnn, optimizer, device, grad_acc, scheduler=scheduler, label_index=label)
              epoch_loss=np.mean(batches_losses_tmp)
              print(f"\n*** avg_loss : {epoch_loss:.2f}, time : ~{(time.time()-t0)//60} min ({time.time()-t0:.2f} sec) ***\n")
              t1=time.time()
              output, target, val_losses_tmp=individual_rnn_eval_loop_fun1(val_data_loader, model_rnn, device)
              print(f"==> evaluation : avg_loss = {np.mean(val_losses_tmp):.2f}, time : {time.time()-t1:.2f} sec\n")
              tmp_evaluate=individual_evaluation(target, output)
              print(f"=====>\t{tmp_evaluate}")
              if tmp_evaluate['max f1 metrics']['f1'] > best_f1:
                    del best_model
                    torch.cuda.empty_cache()
                    best_model = model_rnn
                    best_f1 = tmp_evaluate['max f1 metrics']['f1']
                    best_threshold = tmp_evaluate['max f1 metrics']['threshold']
              val_acc.append(tmp_evaluate['default_accuracy'])
              val_losses.append(val_losses_tmp)
              batches_losses.append(batches_losses_tmp)
              
          del model_rnn, optimizer, scheduler
          print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    torch.save(best_model, f"/research/jamunoz/models/robert_avg/model_"+str(lab)+"_tr_"+str(best_threshold)+".pt")
    del model, bert_model, best_model
    torch.cuda.empty_cache()
    print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx END Para label: ", Y_KEYS[label], " ("+str(label)+") xxxxxxxxxxxxxxxxxxxxxxxxx")

#################################################### Para label:  Apoyo Kinesióloga(o)  (7) ##############
--------------------------------------lr: 1e-07-----------------------------
+++++++++++++++++++++++++++++++++++++++++++++++++++ n layers: 1 ++++++++++++++++++++++++++++++++++++++++++++++++
/////////////////////////////////////////// GA: 16 ///////////////////////////////////////////////////////



=============== EPOCH 1 / 6 ===============

___ batch index = 0 / 1836 (0.00%), loss = 0.8028, time = 0.03 secondes ___


<ipython-input-11-5fb9fe6293fb>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  remain = torch.tensor(remain, dtype=torch.long)
<ipython-input-23-5db6513a9937>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  miniset = [torch.tensor(labels[0])]


___ batch index = 250 / 1836 (13.62%), loss = 0.5594, time = 12.75 secondes ___


___ batch index = 500 / 1836 (27.23%), loss = 0.5765, time = 12.31 secondes ___


...

___ batch index = 1500 / 1836 (81.70%), loss = 0.5359, time = 11.97 secondes ___


___ batch index = 1750 / 1836 (95.32%), loss = 0.5216, time = 13.11 secondes ___



*** avg_loss : 0.57, time : ~1.0 min (91.80 sec) ***



==> evaluation : avg_loss = 0.69, time : 29.54 sec

threshold (g-mean):  1
threshold (f1):  1
=====>	{'Positive Rate': 0.4332784184514003, 'max g_mean metrics': {'threshold': 1, 'accuracy': 0.5667215815485996, 'f1': 0.0, 'recall': 0.0, 'specificity': 1.0, 'g_mean': 0.0}, 'max f1 metrics': {'threshold': 1, 'accuracy': 0.5667215815485996, 'f1': 0.0, 'recall': 0.0, 'specificity': 1.0, 'g_mean': 0.0}, 'default_accuracy': 0.5667215815485996, 'default_f1': 0.0, 'default_recall': 0.0, 'default_specificity': 1.0, 'default_g_mean': 0.0}
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ END @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx END Para label:  Adecuación curricular de objetivos  (13) xxxxxxxxxxxxxxxxxxxxxxxxx
